In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
from torch import optim
from sklearn.preprocessing import LabelEncoder

In [6]:
traindf = pd.read_csv("train.csv",index_col='id')
testdf = pd.read_csv("test.csv")
testdf.head()
lencoder = LabelEncoder()
y = lencoder.fit_transform(traindf['target']).astype('int64')
traindf.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,6,1,0,0,0,0,7,0,0,0,3,0,1,0,0,3,3,1,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,11,0,0,0,0,0,9,0,0,0,0,0,0,3,0,1,3,0,0,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,5,4,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,5,0,1,0,0,0,1,0,0,1,0,7,0,1,1,0,3,4,0,0,1,3,0,2,0,0,8,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,2,0,5,0,0,4,0,0,22,2,1,0,0,0,0,3,0,37,0,1,3,13,0,10,0,3,1,1,0,7,0,0,2,0,1,0,0,0,0,0,0,10,0,0,25,1,0,1,2,0,2,0,7,0,0,0,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_2


In [7]:
print(y)
print(type(y))
traindf['target']=y
traindf.head()

[5 5 1 ... 7 6 7]
<class 'numpy.ndarray'>


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,6,1,0,0,0,0,7,0,0,0,3,0,1,0,0,3,3,1,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,11,0,0,0,0,0,9,0,0,0,0,0,0,3,0,1,3,0,0,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,0,2,0,0,5
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,5
2,0,0,0,0,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,5,4,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,5,0,1,0,0,0,1,0,0,1,0,7,0,1,1,0,3,4,0,0,1,3,0,2,0,0,8,0,0,0,0,1,0,0,0,0,1
3,0,0,7,0,1,5,2,2,0,1,2,0,5,0,0,4,0,0,22,2,1,0,0,0,0,3,0,37,0,1,3,13,0,10,0,3,1,1,0,7,0,0,2,0,1,0,0,0,0,0,0,10,0,0,25,1,0,1,2,0,2,0,7,0,0,0,0,4,0,2,2,0,4,3,0,7
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [8]:
n_input = len(traindf.columns) -1
numClasses = traindf['target'].nunique()
batchSize = 64

In [9]:
print(numClasses)
columns = list(traindf.columns)
columns.pop()

9


'target'

In [10]:
from sklearn.preprocessing import MinMaxScaler 
scaler=MinMaxScaler() 
traindf[columns] =scaler.fit_transform(traindf[columns]) 
traindf.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.0,0.093750,0.014286,0.000000,0.000000,0.000000,0.000000,0.184211,0.000000,0.000000,0.0,0.081081,0.0,0.03125,0.000000,0.0,0.214286,0.136364,0.003802,0.000000,0.060606,0.0,0.0,0.0,0.000000,0.0,0.000000,0.009524,0.000000,0.000000,0.025641,0.0,0.000000,0.000000,0.000000,0.00000,0.323529,0.0,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000,0.0,0.053571,0.000000,0.026316,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0125,0.009804,0.0,0.0,0.055556,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.032787,0.000000,0.0,5
1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.027027,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.009524,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.024390,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.031746,0.000000,0.000000,0.008547,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.000000,0.083333,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.007692,0.0,5
2,0.000000,0.0,0.000000,0.000000,0.000000,0.013158,0.000000,0.100000,0.000000,0.000000,0.030303,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.019011,0.133333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.011905,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.00000,0.147059,0.0,0.102041,0.0,0.013699,0.000000,0.000000,0.000000,0.033333,0.000000,0.0,0.025,0.0,0.125000,0.000000,0.026316,0.027778,0.000000,0.039474,0.086957,0.000000,0.000000,0.002841,0.012987,0.0000,0.019608,0.0,0.0,0.148148,0.000000,0.000000,0.0,0.000000,0.014925,0.0,0.000000,0.000000,0.0,1
3,0.000000,0.0,0.109375,0.000000,0.026316,0.065789,0.046512,0.066667,0.000000,0.013889,0.060606,0.0,0.135135,0.0,0.00000,0.033058,0.0,0.000000,1.000000,0.007605,0.033333,0.000000,0.0,0.0,0.0,0.020134,0.0,0.440476,0.000000,0.011905,0.136364,0.333333,0.0,0.243902,0.000000,0.073171,0.02381,0.029412,0.0,0.142857,0.0,0.000000,0.037736,0.000000,0.037037,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.136986,0.000000,0.000000,0.240385,0.013158,0.000000,0.032258,0.066667,0.000000,0.008658,0.0000,0.068627,0.0,0.0,0.000000,0.000000,0.050633,0.0,0.030769,0.029851,0.0,0.065574,0.023077,0.0,7
4,0.016393,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.033333,0.000000,0.0,0.0,0.0,0.006711,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.058824,0.0,0.102041,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.028846,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1


In [11]:
dataset = torch.tensor(traindf.values)

In [12]:
trainLoader = torch.utils.data.DataLoader(dataset=torch.tensor(traindf.values), batch_size=batchSize, shuffle=True)

In [13]:
learningRate=0.03

In [14]:
model = nn.Sequential(nn.Linear(n_input,32),
                      nn.ReLU(),
                      nn.Linear(32, 16),
                      nn.ReLU(),
                      nn.Linear(16,numClasses),
                      nn.LogSoftmax(dim=1))
criterion=nn.NLLLoss()

optimizer=optim.SGD(model.parameters(),lr=learningRate)

In [15]:
epoch=20
for e in range(epoch):
  running_loss=0
  for i,data in enumerate(trainLoader):
    labels=data[:,-1]
    attributes=data[:,0:n_input].float()
    logits=model(attributes)
    loss= criterion(logits,labels.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  else:
    print(f"Training loss: {running_loss/len(trainLoader)}")

Training loss: 1.9090631348037719
Training loss: 1.8830439376449586
Training loss: 1.8486084342575073
Training loss: 1.8233223584747313
Training loss: 1.8084684272003173
Training loss: 1.7992249631500243
Training loss: 1.7940120401382447
Training loss: 1.7908303713607787
Training loss: 1.7889383882904053
Training loss: 1.7871635175323486
Training loss: 1.7858320505523682
Training loss: 1.7845035958099364
Training loss: 1.7832161221313476
Training loss: 1.7819717590713502
Training loss: 1.780549946937561
Training loss: 1.779547629547119
Training loss: 1.7787233162689209
Training loss: 1.7774418494796753
Training loss: 1.7768847751998902
Training loss: 1.7758474392700194


In [16]:
testdf[columns]=scaler.fit_transform(testdf[columns])

In [17]:
testdfTorch = torch.tensor(testdf.drop(['id'], axis=1).values)
with torch.no_grad():
    test_preds = model(testdfTorch.float())
    test_preds = torch.exp(test_preds)

In [18]:
test_preds[0]
torch.sum(test_preds[0])

tensor(1.0000)

In [19]:
output = pd.DataFrame({'id': testdf.id,
                       'Class_1': test_preds[:, 0],
                       'Class_2': test_preds[:, 1],
                       'Class_3': test_preds[:, 2],
                       'Class_4': test_preds[:, 3],
                       'Class_5': test_preds[:, 4],
                       'Class_6': test_preds[:, 5],
                       'Class_7': test_preds[:, 6],
                       'Class_8': test_preds[:, 7],
                       'Class_9': test_preds[:, 8],
                      })

output.to_csv('submission_tps.csv', index=False)